In [26]:
import bokeh.plotting as bpl
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import tifffile as tif
from time import time, sleep

from PIL import Image

try:
    cv2.setNumThreads(0)
except():
    pass

try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes
        # when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour

from caiman.source_extraction import cnmf as cnmf
from scipy.sparse import csc_matrix

from scipy.io import savemat, loadmat

from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.interpolation import shift

from caiman.source_extraction.cnmf.utilities import extract_DF_F, detrend_df_f

import pandas as pd

import h5py
bpl.output_notebook()


SyntaxError: invalid syntax (<ipython-input-26-a78c9f78105c>, line 10)

In [28]:
arr = np.random.randint(255,size=(256,256)) - 128
arr_type = arr.dtype
new_size = (128,128)
im = Image.fromarray(arr.astype('float')).resize(new_size, resample=Image.BILINEAR)
new_arr = np.array(im)
new_arr

array([[ 22.22449  , -44.535713 ,   7.214286 , ..., -21.589285 ,
        -27.446428 ,  12.755103 ],
       [ 15.214287 , -48.046875 ,   7.78125  , ...,   2.515625 ,
         -1.375    ,   4.589286 ],
       [ 29.428572 ,  13.828125 ,  -0.796875 , ...,  43.546875 ,
         33.375    ,   8.482143 ],
       ...,
       [-25.571428 ,  21.59375  ,   0.609375 , ...,  18.75     ,
         11.390625 ,   5.6250005],
       [ 18.821428 , -33.796875 ,   1.140625 , ...,  18.96875  ,
         11.       ,  19.839285 ],
       [ 22.081633 ,   9.535714 , -33.339287 , ...,  33.732143 ,
        -31.535715 , -18.367348 ]], dtype=float32)

In [93]:
# define utility functions

def resize_frames(image_in, new_frame_size, resample = Image.BILINEAR, dtype_out = np.float32):
    """
    resize image stack, possibly with multiple channels
    
    params
    image_in: input stack. can be any shape, but the last 2 dimensions must be the image dimensions.
    new_frame_size: length-2 tuple of the new image dimensions
    resample: PIL.Image resample type; default is PIL.Image.BILINEAR
    dtype_out: data type of output
    
    output
    image stack of type dtype_out with last 2 dimensions = new_frame_size and all other
        dimensions equal to those of image_in
    """
    im_dims = image_in.shape
    frame_dims = im_dims[-2:]
    stack_dims = im_dims[:-2]
    image_in = np.reshape(image_in, (-1,) + frame_dims, order='C')
    image_out = np.empty((len(image_in),) + new_frame_size)
    for i,frame in enumerate(image_in):
        image_out[i] = np.array( Image.fromarray(frame.astype(np.float32)) \
                                .resize(new_frame_size, resample=resample) )
    return image_out.astype(dtype_out).reshape(stack_dims + new_frame_size)


def bin_frames(arr, bin_size, axis=0, op=np.mean, dtype_out=np.float32):
    """
    Group together consecutive image frames from a stack and combine frames within groups via a given operation
    
    params
    arr: input image stack (allowed to have channels). the only requirements are that the image dimensions
        are the last 2 dimensions, and 'axis' is not one of the image dimensions.
    bin_size: size of binning groups
    axis: bin axis
    op: operation used to combine binned frames. should be a numpy array operation; default is np.mean
    dtype_out: output data type
    
    output:
    array binned along axis, of data type dtype_out
    """
    in_shape = arr.shape
    arr = arr.swapaxes(axis,0) # get "axis" to the front of the dimensions
    ax_length = len(arr)
    if ax_length % bin_size:
        raise Exception("the length of arr along axis must be a multiple of bin_size")
    arr = arr.reshape((-1,bin_size) + arr.shape[1:], order='C') # front is now (num_bins) x (bin_size)
    output = op(arr, axis=1, keepdims=False) # aggregate along the bin_size dimension
    output = output.swapaxes(axis,0) # return axis to appropriate position
    return output.astype(dtype_out)
    

In [73]:
mov = tif.imread('/gpfs/home/stetlb01/sample_movie/sample_movie.tif')
mov2 = resize_images(mov, (256,256))

In [90]:
mov_mean = bin_frames(mov2, bin_size=5)
mov_max = bin_frames(mov2, bin_size=5, op=np.max)

In [94]:
mov_mean.shape

(40, 256, 256)